In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ======================== CELL 1: DEPENDENCIES ==========================
import subprocess
import sys

print('🔧 Installing dependencies...')
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], capture_output=True, check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyarrow==15.0.2", "keybert", "rank-bm25", "faiss-cpu", "sacremoses"], check=True)
print('✅ Dependencies installed. RESTART kernel and run Cell 2.')


In [1]:
# ===================== CELL 2: IMPORTS & SETUP =====================
import warnings
warnings.filterwarnings("ignore")

import re
import json
import time
import pickle
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import torch
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize, sent_tokenize
from rank_bm25 import BM25Okapi

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# ============================================================================
# CONFIGURATION FOR MENTAL HEALTH DOMAIN
# ============================================================================

@dataclass
class DomainConfig:
    name: str = "mental_health"
    dataset_name: str = "Amod/mental_health_counseling_conversations"
    dataset_split: str = "train"
    chunk_window: int = 3
    chunk_stride: int = 1
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"

config = DomainConfig()

print(f"📋 Building index for: {config.name}")
print(f"📦 Dataset: {config.dataset_name}")


2025-11-01 18:18:11.891146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762021092.097509     107 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762021092.149730     107 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔧 Using device: cuda
📋 Building index for: mental_health
📦 Dataset: Amod/mental_health_counseling_conversations


In [2]:
# ===================== CELL 3: DATASET LOADING =====================

def extract_qa_pairs(dataset) -> list:
    """Extract Q&A pairs from mental health dataset."""
    qa_data = []
    
    for idx, row in enumerate(dataset):
        try:
            # Mental health dataset structure
            if 'Context' in row and 'Response' in row:
                qa_data.append({
                    "question": str(row['Context']).strip(),
                    "answer": str(row['Response']).strip(),
                    "source_id": idx
                })
            # Alternative structure
            elif 'question' in row and 'answer' in row:
                qa_data.append({
                    "question": str(row['question']).strip(),
                    "answer": str(row['answer']).strip(),
                    "source_id": idx
                })
            # Conversation format
            elif 'conversations' in row:
                conversations = row['conversations']
                if isinstance(conversations, list) and len(conversations) >= 2:
                    question = conversations[0].get('value', '') if isinstance(conversations[0], dict) else str(conversations[0])
                    answer = conversations[1].get('value', '') if isinstance(conversations[1], dict) else str(conversations[1])
                    if question and answer:
                        qa_data.append({
                            "question": question.strip(),
                            "answer": answer.strip(),
                            "source_id": idx
                        })
        except Exception as e:
            if idx < 3:
                print(f"⚠️ Row {idx} skipped: {e}")
            continue
    
    return qa_data

# Load dataset
print(f"📥 Loading {config.dataset_name}...")
dataset = load_dataset(config.dataset_name, split=config.dataset_split)
print(f"✅ Loaded {len(dataset)} rows")

# Extract Q&A pairs
qa_data = extract_qa_pairs(dataset)
print(f"✅ Extracted {len(qa_data)} Q&A pairs")

if len(qa_data) < 100:
    print(f"⚠️ WARNING: Only {len(qa_data)} pairs extracted!")
    print(f"Sample row structure: {dataset[0]}")


📥 Loading Amod/mental_health_counseling_conversations...


README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

✅ Loaded 3512 rows
✅ Extracted 3512 Q&A pairs


In [3]:
# ===================== CELL 4: TEXT CHUNKING =====================

def create_chunks(data: List[Dict], window: int = 3, stride: int = 1) -> List[Dict]:
    """Create sentence-level chunks from answers."""
    chunks = []
    
    for item in data:
        text = item.get("answer", "")
        if not text or len(text) < 50:
            continue
        
        sentences = sent_tokenize(text)
        if not sentences:
            continue
        
        # If answer is short, keep as one chunk
        if len(sentences) <= window:
            chunks.append({
                "chunk": " ".join(sentences),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
            continue
        
        # Create overlapping chunks
        for i in range(0, max(1, len(sentences) - window + 1), stride):
            chunks.append({
                "chunk": " ".join(sentences[i:i + window]),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
    
    return chunks

# Create chunks
print(f"🔪 Creating chunks (window={config.chunk_window}, stride={config.chunk_stride})...")
chunks = create_chunks(qa_data, window=config.chunk_window, stride=config.chunk_stride)
print(f"✅ Created {len(chunks)} chunks")

# Sample chunks
print("\n📝 Sample chunks:")
for i, chunk in enumerate(chunks[:3]):
    print(f"{i+1}. {chunk['chunk'][:150]}...")


🔪 Creating chunks (window=3, stride=1)...
✅ Created 22565 chunks

📝 Sample chunks:
1. If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is ...
2. Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so ...
3. Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all...


In [4]:
# ===================== CELL 5: BUILD FAISS INDEX =====================

# Load embedder
print(f"📦 Loading embedder: {config.embed_model}")
embedder = SentenceTransformer(config.embed_model, device=device)
print(f"✅ Embedder loaded")

# Extract chunk texts
id2doc = [chunk["chunk"] for chunk in chunks]
print(f"📊 Encoding {len(id2doc)} chunks...")

# Create embeddings
embeddings = embedder.encode(
    id2doc,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=64,
    convert_to_numpy=True
).astype('float32')

print(f"✅ Embeddings shape: {embeddings.shape}")

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print(f"✅ FAISS index built: {index.ntotal} vectors, dimension {dim}")


📦 Loading embedder: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedder loaded
📊 Encoding 22565 chunks...


Batches:   0%|          | 0/353 [00:00<?, ?it/s]

✅ Embeddings shape: (22565, 384)
✅ FAISS index built: 22565 vectors, dimension 384


In [5]:
# ===================== CELL 6: BUILD BM25 INDEX =====================

# Tokenize for BM25
print(f"🔨 Building BM25 index...")
bm25_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
bm25 = BM25Okapi(bm25_corpus)
print(f"✅ BM25 index built")


🔨 Building BM25 index...
✅ BM25 index built


In [6]:
# ===================== CELL 7: SAVE INDEXES =====================

# File names
index_file = f"{config.name}_faiss.index"
id2doc_file = f"{config.name}_id2doc.pkl"
metadata_file = f"{config.name}_metadata.json"

# Save FAISS index
print(f"💾 Saving FAISS index to {index_file}...")
faiss.write_index(index, index_file)

# Save id2doc mapping
print(f"💾 Saving id2doc to {id2doc_file}...")
with open(id2doc_file, "wb") as f:
    pickle.dump(id2doc, f)

# Save metadata
metadata = {
    "created_at": time.time(),
    "domain": config.name,
    "dataset": config.dataset_name,
    "n_vectors": int(index.ntotal),
    "embedding_dim": dim,
    "chunk_window": config.chunk_window,
    "chunk_stride": config.chunk_stride,
    "embed_model": config.embed_model
}

print(f"💾 Saving metadata to {metadata_file}...")
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*80)
print("✅ INDEX BUILDING COMPLETE!")
print("="*80)
print(f"📁 Files created:")
print(f"  - {index_file}")
print(f"  - {id2doc_file}")
print(f"  - {metadata_file}")
print(f"\n📊 Statistics:")
print(f"  - Total vectors: {index.ntotal}")
print(f"  - Embedding dimension: {dim}")
print(f"  - Source Q&A pairs: {len(qa_data)}")
print(f"  - Chunks created: {len(chunks)}")
print("="*80)


💾 Saving FAISS index to mental_health_faiss.index...
💾 Saving id2doc to mental_health_id2doc.pkl...
💾 Saving metadata to mental_health_metadata.json...

✅ INDEX BUILDING COMPLETE!
📁 Files created:
  - mental_health_faiss.index
  - mental_health_id2doc.pkl
  - mental_health_metadata.json

📊 Statistics:
  - Total vectors: 22565
  - Embedding dimension: 384
  - Source Q&A pairs: 3512
  - Chunks created: 22565


In [7]:
# ===================== CELL 8: VERIFY INDEX (OPTIONAL) =====================

def test_retrieval(query: str, top_k: int = 5):
    """Test the built index with a sample query."""
    print(f"\n🔍 Testing query: '{query}'")
    
    # Embed query
    query_emb = embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
    
    # Search FAISS
    D, I = index.search(query_emb, top_k)
    
    print(f"\n📋 Top {top_k} results:")
    for i, (idx, score) in enumerate(zip(I[0], D[0]), 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   {id2doc[idx][:200]}...")

# Test queries
test_queries = [
    "I feel anxious and depressed",
    "How to deal with stress?",
    "I'm having panic attacks"
]

print("\n" + "="*80)
print("🧪 TESTING INDEX")
print("="*80)

for query in test_queries:
    test_retrieval(query, top_k=3)



🧪 TESTING INDEX

🔍 Testing query: 'I feel anxious and depressed'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.6048
   I am very sorry that you are struggling. I have a few thoughts and perhaps some of my colleagues will add something else. Depression and anxiety can both be chronic conditions, as you have experienced...

2. Score: 0.6048
   I am very sorry that you are struggling. I have a few thoughts and perhaps some of my colleagues will add something else. Depression and anxiety can both be chronic conditions, as you have experienced...

3. Score: 0.5910
   You are not alone, and many people are experiencing very similar
anxieties. A Life Coach or professional therapist could help to guide you
through these emotions and refer you to additional profession...

🔍 Testing query: 'How to deal with stress?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.7620
   So, instead you need to find ways to help live with the stressors that you already have. They can be things like creating a more manageable schedule, introducing healthy eating or exercising, financia...

2. Score: 0.7604
   Try to take care
of yourself in other ways, such as eating well. Exercise can really help when
we get stressed. I hope some of this was helpful....

3. Score: 0.7604
   Try to take care
of yourself in other ways, such as eating well. Exercise can really help when
we get stressed. I hope some of this was helpful....

🔍 Testing query: 'I'm having panic attacks'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.6304
   Remember that I'm not asking you to actually diagnose yourself with one of these things, but just offering some basic information that may help you be able to talk about what is going on.If you are ha...

2. Score: 0.6304
   Remember that I'm not asking you to actually diagnose yourself with one of these things, but just offering some basic information that may help you be able to talk about what is going on.If you are ha...

3. Score: 0.6271
   Anxiety or panic attacks can be very frightening. Here are a few "tools" you can use that will help in the short term:  Keeping your mind occupied by listening to books on tape may help;  Counting bac...
